In [65]:
!nvidia-smi

Mon Jan 17 08:53:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro M4000        On   | 00000000:00:05.0 Off |                  N/A |
| 56%   61C    P0    45W / 120W |   7730MiB /  8126MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [73]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from tqdm import tqdm
from zipfile import ZipFile
from CustomLayers import CustomConv2D, CustomConvTranspose2D

In [74]:
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# physical_devices

In [75]:
# try:
#   os.makedirs("celeba_gan")
# except:
#   pass

# url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
# path = "celeba_gan/data.zip"
# gdown.download(url, path, quiet = True)

# with ZipFile("celeba_gan/data.zip", "r") as zipobj:
#   zipobj.extractall("celeba_gan")

In [51]:
data_directory = "celeba_gan/img_align_celeba"
dataset = keras.preprocessing.image_dataset_from_directory(directory = data_directory,
                                                           label_mode = None,
                                                           image_size = (128, 128),
                                                           batch_size = 64,
                                                           shuffle = True).map(lambda x : x/255.0)

Found 202599 files belonging to 1 classes.


### **Generator**

In [76]:
noise_dim = 128
input = layers.Input(shape = (noise_dim,))
dense = layers.Dense(4 * 4 * 1024)(input)
conv = layers.Reshape((4, 4, 1024))(dense)
conv_t = CustomConvTranspose2D(512, kernel_size = 5, strides = 2, padding = "same")(conv)

conv_t = layers.Conv2DTranspose(256, kernel_size = 5, strides = 2, padding = "same")(conv_t)
conv_t = layers.Conv2DTranspose(128, kernel_size = 5, strides = 2, padding = "same")(conv_t)
              
conv_t = layers.Conv2DTranspose(64, kernel_size = 5, strides = 2, padding = "same")(conv_t)
output = layers.Conv2DTranspose(3, kernel_size = 5, strides = 2, padding = "same", activation = "sigmoid")(conv_t)

generator = keras.Model(inputs = input, outputs = output)
print(generator.summary())

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
dense_13 (Dense)             (None, 16384)             2113536   
_________________________________________________________________
reshape_7 (Reshape)          (None, 4, 4, 1024)        0         
_________________________________________________________________
custom_conv_transpose2d_4 (C (None, 8, 8, 512)         13109760  
_________________________________________________________________
conv2d_transpose_36 (Conv2DT (None, 16, 16, 256)       3277056   
_________________________________________________________________
conv2d_transpose_37 (Conv2DT (None, 32, 32, 128)       819328    
_________________________________________________________________
conv2d_transpose_38 (Conv2DT (None, 64, 64, 64)        2048

### **Discriminator**

In [77]:
input = keras.Input(shape = (128, 128, 3))
conv = CustomConv2D(64, kernel_size = 4, strides = 2, padding = "same")(input)
conv = CustomConv2D(128, kernel_size = 4, strides = 2, padding = "same")(conv)
conv = CustomConv2D(256, kernel_size = 4, strides = 2, padding = "same")(conv)
output = layers.Flatten()(conv)
output = layers.Dropout(0.2)(output)
output = layers.Dense(1, activation = "sigmoid")(output)

discriminator = keras.Model(inputs = input, outputs = output)
print(discriminator.summary())

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
custom_conv2d_7 (CustomConv2 (None, 64, 64, 64)        3392      
_________________________________________________________________
custom_conv2d_8 (CustomConv2 (None, 32, 32, 128)       131712    
_________________________________________________________________
custom_conv2d_9 (CustomConv2 (None, 16, 16, 256)       525568    
_________________________________________________________________
flatten_6 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 65536)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 6553

### **Optimisers**

In [78]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

### **Train Discriminator**

In [ ]:
from IPython.display import clear_output
try:
    os.mkdir("models")
except:
    pass
try:
    generator = tf.keras.models.load_model(os.path.join("models", "generator"))
    discriminator = tf.keras.models.load_model(os.path.join("models", "discriminator"))
    print("Loading Models from Disk!")
except:
    print("Starting Training from Scratch!")

for epoch in range(10):
    for idx, real in enumerate(tqdm(dataset)):
        batch_size = real.shape[0]
        with tf.GradientTape(persistent = True) as tape:
            fake = generator(random_latent_vectors)
            output_disc_fake = discriminator(fake)
            output_disc_real = discriminator(real)
            
            loss_disc_real = loss_fn(tf.ones(batch_size, 1), output_disc_real)
            loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), output_disc_fake)
            loss_disc = loss_disc_real + loss_disc_fake            
            loss_gen = loss_fn(tf.ones(batch_size, 1), output_disc_fake)
        
        if idx % 100 == 0:
            img = fake[0].numpy().astype('float32')
            result_name = f"{epoch}_{idx}_.png"
            result_path = "results"
            try:
                os.mkdir(result_path)
            except:
                pass
            result_path = os.path.join(result_path, result_name)
            plt.imsave(result_path, img)
            generator.save(os.path.join("models", "generator"))
            discriminator.save(os.path.join("models", "discriminator"))
            clear_output()

        grads = tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(zip(grads, discriminator.trainable_weights))

        grads = tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(zip(grads, generator.trainable_weights))

 19%|█▉        | 600/3166 [07:58<32:28,  1.32it/s]  

INFO:tensorflow:Assets written to: models/generator/assets


INFO:tensorflow:Assets written to: models/generator/assets
